# Tuesday Health: Hospice Quality Metrics Technical Assessment
Author: Kyle Biron  

In [ ]:
import duckdb
import numpy
import pandas as pd
from pprint import pprint

# configure pandas to show full string values and all rows/columns
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# set duckdb directory path
db_dir = "/Users/kyle/Documents/personal_projects/repos/kyle-tuesday-health-tech-assignment/hospice_quality_metrics/db"

# connect to an existing duckdb database
con = duckdb.connect(f'{db_dir}/hospice_quality_metrics.db')

# list and print all tables in the connected duckdb database
tables = con.execute("SHOW TABLES").df()
print(tables)

In [ ]:
# Query the database
stg_hospice_provider_df = con.execute("SELECT * FROM stg_hospice_provider limit 5").df()

# Display the dataframe
display(stg_hospice_provider_df)

In [ ]:
con.execute("SELECT distinct measure_name FROM stg_hospice_provider").df()

In [ ]:
con.execute("SELECT *  FROM stg_hospice_provider where measure_name = 'Per-beneficiary spending (U.S. dollars $)' limit 5").df()




In [20]:
con.close()